In [4]:
%%capture
%run ./index.ipynb

## Repair Operator 

A simple approach is to handle constraints through a repair function. This is only possible if the equation of the constraint is known. The repair makes sure every solution that is evaluated is, in fact, feasible. Let us consider the equality constraint example.

Let us define a `Repair` operator that always satisfies the equality constraint (the inequality constraint is simply ignored and will be figured out by the algorithm)

In [10]:
from pymoo.core.repair import Repair

class MyRepair(Repair):

    def _do(self, problem, X, **kwargs):
        X[:, 0] = 1/3 * X[:, 1]
        return X

Now the algorithm object needs to be initialized with the `Repair` operator and then can be run to solve the problem:

In [11]:
from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.optimize import minimize

algorithm = GA(repair=MyRepair())

res = minimize(ConstrainedProblemWithEquality(),
               algorithm,
               ('n_gen', 20),
               seed=1,
               verbose=True)

print("Best solution found: \nX = %s\nF = %s\nCV = %s" % (res.X, res.F, res.CV))

n_gen  |  n_eval  |     cv_min    |     cv_avg    |     f_avg     |     f_min    
     1 |      100 |  0.000000E+00 |  0.3516478340 |  1.1816456287 |  1.0001924199
     2 |      200 |  0.000000E+00 |  0.0206023997 |  1.1675620969 |  1.0001924199
     3 |      300 |  0.000000E+00 |  0.000000E+00 |  1.1107388988 |  1.0001924199
     4 |      400 |  0.000000E+00 |  0.000000E+00 |  1.0524340820 |  1.0001875222
     5 |      500 |  0.000000E+00 |  0.000000E+00 |  1.0306380519 |  1.0001875222
     6 |      600 |  0.000000E+00 |  0.000000E+00 |  1.0172517666 |  1.0001875222
     7 |      700 |  0.000000E+00 |  0.000000E+00 |  1.0109417735 |  1.0001875222
     8 |      800 |  0.000000E+00 |  0.000000E+00 |  1.0072225067 |  1.0000905808
     9 |      900 |  0.000000E+00 |  0.000000E+00 |  1.0049397512 |  1.0000460954
    10 |     1000 |  0.000000E+00 |  0.000000E+00 |  1.0040643895 |  1.0000460954
    11 |     1100 |  0.000000E+00 |  0.000000E+00 |  1.0032576344 |  1.0000460954
    12 |     120

If you would like to compare the solution without a repair you will see how searching only in the feasible space helps:

In [13]:
from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.optimize import minimize


algorithm = GA()

res = minimize(ConstrainedProblemWithEquality(),
               algorithm,
               ('n_gen', 20),
               seed=1,
               verbose=True)

print("Best solution found: \nX = %s\nF = %s\nCV = %s" % (res.X, res.F, res.CV))

n_gen  |  n_eval  |     cv_min    |     cv_avg    |     f_avg     |     f_min    
     1 |      100 |  0.0168997810 |  1.1966219666 |             - |             -
     2 |      200 |  0.0006806481 |  0.5277543787 |             - |             -
     3 |      300 |  0.0006806481 |  0.2246605969 |             - |             -
     4 |      400 |  0.0001335846 |  0.0831058535 |             - |             -
     5 |      500 |  0.0001335846 |  0.0417134481 |             - |             -
     6 |      600 |  0.0001335846 |  0.0255767204 |             - |             -
     7 |      700 |  0.0001335846 |  0.0163322959 |             - |             -
     8 |      800 |  0.0001335846 |  0.0122030304 |             - |             -
     9 |      900 |  0.0001335846 |  0.0100974433 |             - |             -
    10 |     1000 |  0.0001335846 |  0.0079568679 |             - |             -
    11 |     1100 |  0.0001335846 |  0.0055451925 |             - |             -
    12 |     120